<a href="https://colab.research.google.com/github/mauricio-ms/motor-imagery-convolutional-recurrent-neural-network/blob/master/notebooks/cnn-1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verify if GPU is enabled

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
  raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


Mount Google Drive directory

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install needed packages

In [3]:
!pip install colorlog

In [4]:
import logging
import colorlog


def get_logger(name="__main__", debug=False):
    log_format = (
        "[%(levelname)s] "
        "%(asctime)s - "
        "%(name)s: "
        "%(funcName)s - "
        "%(message)s"
    )
    bold_seq = "\033[1m"
    colorlog_format = (
        f"{bold_seq} "
        "%(log_color)s "
        f"{log_format}"
    )
    colorlog.basicConfig(format=colorlog_format)

    logger = logging.getLogger(name)
    if debug:
        logger.setLevel(logging.DEBUG)
    else:
        logger.setLevel(logging.INFO)

    # Output full log
    fh = logging.FileHandler("app.log")
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter(log_format)
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    return logger

In [5]:
import os
import numpy as np

LOGGER = get_logger("physionet.py")
TFRECORD_FILES_DIR = "drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/preprocessed-tfrecord-files"

In [6]:
def load_data(train_size=0.75, validation_size=None, n_subjects=None,
              train_subjects=None, test_subjects=None, **kwargs):
    LOGGER.info("Loading Physionet dataset ...")
    subjects = np.array(sorted(os.listdir(TFRECORD_FILES_DIR)))
    if n_subjects is not None:
        np.random.shuffle(subjects)
        subjects = subjects[:n_subjects]

    if train_subjects is None or test_subjects is None:
        train_subjects, test_subjects = _train_test_split_subjects(subjects, train_size)

    if validation_size is not None:
        train_subjects, validation_subjects = _train_test_split_subjects(train_subjects, 1-validation_size)
        LOGGER.info(f"(Train, Validation, Test) Subjects = "
                    f"({len(train_subjects)}, {len(validation_subjects)}, {len(test_subjects)})")
        LOGGER.info(f"Train subjects: {train_subjects}")
        LOGGER.info(f"Validation subjects: {validation_subjects}")
        LOGGER.info(f"Test subjects: {test_subjects}")
        return _load_set(train_subjects, **kwargs), \
            _load_set(validation_subjects, **kwargs), \
            _load_set(test_subjects, **kwargs)

    LOGGER.info(f"(Train, Test) Subjects = ({len(train_subjects)}, {len(test_subjects)})")
    LOGGER.info(f"Train subjects: {train_subjects}")
    LOGGER.info(f"Test subjects: {test_subjects}")
    return _load_set(train_subjects, **kwargs), _load_set(test_subjects, **kwargs)


def _train_test_split_subjects(subjects, train_size):
    train_subjects_mask = np.random.rand(len(subjects)) < train_size
    return subjects[train_subjects_mask], subjects[~train_subjects_mask]


# TODO - convert_to_2d should define the data directory
def _load_set(subjects, n_readers=tf.data.experimental.AUTOTUNE,
              n_parse_threads=tf.data.experimental.AUTOTUNE,
              batch_size=100, convert_to_2d=False, expand_dim=False):
    path_files = [os.path.join(TFRECORD_FILES_DIR, subject, file_name)
                  for subject in subjects
                  for file_name in sorted(os.listdir(os.path.join(TFRECORD_FILES_DIR, subject)))]
    dataset = tf.data.Dataset.list_files(path_files)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TFRecordDataset(filepath, compression_type="GZIP"),
        cycle_length=n_readers, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda r: _preprocess(r, expand_dim=expand_dim),
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.cache()
    return dataset.prefetch(1)


@tf.function
def _preprocess(serialized_eeg_records, expand_dim=False):
    n_channels = 64
    feature_description = {
        "X": tf.io.FixedLenFeature([n_channels], tf.float32),
        "y": tf.io.FixedLenFeature([], tf.int64)
    }
    parsed_eeg_records = tf.io.parse_example(serialized_eeg_records, feature_description)
    X = parsed_eeg_records["X"]
    y = parsed_eeg_records["y"]
    if expand_dim:
        X = X[..., np.newaxis]
    return X, y

In [7]:
train_set, test_set = load_data(expand_dim=True)

  [INFO] 2020-08-18 00:22:46,412 - physionet.py: load_data - Loading Physionet dataset ...
  [INFO] 2020-08-18 00:22:46,419 - physionet.py: load_data - (Train, Test) Subjects = (75, 33)
  [INFO] 2020-08-18 00:22:46,423 - physionet.py: load_data - Train subjects: ['S001' 'S002' 'S003' 'S004' 'S005' 'S007' 'S009' 'S010' 'S011' 'S012'
 'S013' 'S016' 'S017' 'S018' 'S019' 'S022' 'S023' 'S024' 'S025' 'S027'
 'S029' 'S030' 'S031' 'S032' 'S033' 'S037' 'S040' 'S041' 'S043' 'S045'
 'S046' 'S049' 'S050' 'S051' 'S052' 'S054' 'S055' 'S056' 'S057' 'S058'
 'S060' 'S061' 'S062' 'S064' 'S065' 'S066' 'S068' 'S069' 'S072' 'S073'
 'S075' 'S077' 'S078' 'S081' 'S082' 'S083' 'S084' 'S085' 'S086' 'S090'
 'S091' 'S092' 'S093' 'S096' 'S097' 'S098' 'S099' 'S100' 'S101' 'S102'
 'S103' 'S104' 'S105' 'S106' 'S109']
  [INFO] 2020-08-18 00:22:46,426 - physionet.py: load_data - Test subjects: ['S006' 'S008' 'S014' 'S015' 'S020' 'S021' 'S026' 'S028' 'S034' 'S035'
 'S036' 'S038' 'S039' 'S042' 'S044' 'S047' 'S048' 'S053'

In [13]:
from tensorflow import keras

with tf.device("/device:GPU:0"):
  model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  optimizer = keras.optimizers.Adam(lr=1e-5)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

In [15]:
LOGGER.info("Starting training CNN 1D model ...")
model.fit(train_set, epochs=1, verbose=2)
model.save_weights("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/cnn-1d-weights-until-epoch-10.h5")
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-18 00:55:04,952 - physionet.py: <module> - Starting training CNN 1D model ...


95522/95522 - 428s - loss: 1.2805 - accuracy: 0.5344


  [INFO] 2020-08-18 01:02:14,442 - physionet.py: <module> - Training CNN 1D model end!


In [17]:
from tensorflow import keras

with tf.device("/device:GPU:0"):
  new_model = keras.models.Sequential([
      keras.layers.Conv1D(32, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME",
                          input_shape=[64, 1]),
      keras.layers.Conv1D(64, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Conv1D(128, 3, activation="relu",
                          kernel_initializer="he_normal",
                          padding="SAME"),
      keras.layers.Flatten(),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1024, activation="relu",
                        kernel_initializer="he_normal"),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(5, activation="softmax")
  ])

  new_model.load_weights("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/w1.h5")
  optimizer = keras.optimizers.Adam(lr=1e-5)
  new_model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])

In [18]:
LOGGER.info("Starting training CNN 1D model ...")
new_model.fit(train_set, epochs=1, verbose=2)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-18 01:03:05,541 - physionet.py: <module> - Starting training CNN 1D model ...


95522/95522 - 429s - loss: 1.2675 - accuracy: 0.5352


  [INFO] 2020-08-18 01:10:15,042 - physionet.py: <module> - Training CNN 1D model end!


In [15]:
model.save('drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/saved_model/my_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


  [WARNING] 2020-08-17 22:25:04,310 - tensorflow: new_func - From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


  [WARNING] 2020-08-17 22:25:04,331 - tensorflow: new_func - From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/saved_model/my_model/assets


  [INFO] 2020-08-17 22:25:05,545 - tensorflow: copy_assets_to_destination_dir - Assets written to: drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/saved_model/my_model/assets


In [ ]:
train_subjects = ['S002', 'S006', 'S007', 'S008', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015',
    'S016', 'S017', 'S018', 'S019', 'S021', 'S023', 'S024', 'S025', 'S026', 'S027',
    'S028', 'S029', 'S030', 'S031', 'S032', 'S034', 'S035', 'S036', 'S037', 'S038',
    'S039', 'S040', 'S041', 'S042', 'S043', 'S045', 'S046', 'S047', 'S049', 'S050',
    'S051', 'S053', 'S056', 'S058', 'S059', 'S061', 'S062', 'S063', 'S064', 'S065',
    'S066', 'S067', 'S069', 'S071', 'S072', 'S073', 'S075', 'S076', 'S077', 'S078',
    'S079', 'S080', 'S081', 'S082', 'S083', 'S085', 'S086', 'S087', 'S088', 'S090',
    'S093', 'S094', 'S095', 'S096', 'S097', 'S098', 'S100', 'S102', 'S104', 'S105',
    'S106', 'S107', 'S108']
test_subjects = ['S001', 'S003', 'S004', 'S005', 'S009', 'S020', 'S022', 'S033', 'S044', 'S048',
    'S052', 'S054', 'S055', 'S057', 'S060', 'S068', 'S070', 'S074', 'S084', 'S091',
    'S092', 'S099', 'S101', 'S103', 'S109']
train_set, test_set = load_data(expand_dim=True,
                                train_subjects=train_subjects,
                                test_subjects=test_subjects)

In [12]:
LOGGER.info("Continue training CNN 1D model ...")
model_loaded = keras.models.load_model("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/cnn_1d_TEST.h5")

with tf.device("/device:GPU:0"):
  optimizer = keras.optimizers.Adam(lr=1e-4)
  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=optimizer,
                metrics=["accuracy"])
  
model_loaded.fit(train_set, epochs=1, verbose=2)
#model.save("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/cnn_1d_until_epoch_20.h5")
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-17 21:57:37,032 - physionet.py: <module> - Continue training CNN 1D model ...


119598/119598 - 543s - loss: 1.3306 - accuracy: 0.9901


  [INFO] 2020-08-17 22:06:40,850 - physionet.py: <module> - Training CNN 1D model end!


In [12]:
LOGGER.info("Continue training CNN 1D model ...")
model_loaded = keras.models.load_model("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/cnn_1d_TEST2.h5")
model_loaded.fit(train_set, initial_epoch=1, epochs=2, verbose=2)
#model.save("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/cnn_1d_until_epoch_20.h5")
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-18 00:38:54,817 - physionet.py: <module> - Continue training CNN 1D model ...


Epoch 2/2
95522/95522 - 386s - loss: nan - accuracy: 1.0000


  [INFO] 2020-08-18 00:45:21,206 - physionet.py: <module> - Training CNN 1D model end!


In [9]:
LOGGER.info("Continue training CNN 1D model ...")
new_model = tf.keras.models.load_model("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/saved_model/my_model")
new_model.fit(train_set, epochs=1, verbose=2)
LOGGER.info("Training CNN 1D model end!")

  [INFO] 2020-08-18 00:12:53,124 - physionet.py: <module> - Continue training CNN 1D model ...


108103/108103 - 531s - loss: 1.3181 - accuracy: 0.9885


  [INFO] 2020-08-18 00:21:47,741 - physionet.py: <module> - Training CNN 1D model end!


In [23]:
new_model = tf.keras.models.load_model("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/saved_model/my_model")
new_model.get_weights()

[array([[[ 1.2479185 , -1.0484396 ,  0.22399257, -0.14632225,
           0.6235714 ,  0.16572039, -0.5934104 ,  0.13705234,
          -0.3439609 , -0.97033054, -0.58492875,  0.42525834,
          -0.7948191 ,  0.27080575, -0.51774174,  0.6568736 ,
          -0.48904008, -0.52082247,  0.94049704, -0.35200733,
          -0.17433544,  0.57935137,  0.97350436, -0.6188527 ,
          -0.33460578, -1.0815489 , -1.5792785 ,  0.9739131 ,
          -0.5957952 ,  0.4966284 ,  0.3782153 ,  0.9370097 ]],
 
        [[-0.45444685,  0.38426158, -0.2099472 , -0.93000185,
           0.38673112, -0.7111149 ,  1.1865135 ,  0.2965222 ,
           0.35758755,  0.1236002 , -0.06522775, -0.52790517,
           0.0904016 ,  0.90957785, -0.30399594,  0.42841777,
          -0.58279455,  0.03399912, -0.83413965, -0.62211716,
          -0.35138384, -0.17280042, -0.43189576,  0.17056772,
           0.51769215, -0.05811964,  1.2582767 ,  0.04461861,
          -1.567987  ,  0.5230962 , -0.08693099,  0.534109  ]],
 


In [12]:
model.evaluate(test_set)

31783/31783 [==============================] - 101s 3ms/step - loss: 4.4890 - accuracy: 0.9649


[4.488966941833496, 0.96487957239151]

In [5]:
from tensorflow import keras

model = keras.models.load_model("drive/My Drive/motor-imagery-convolutional-recurrent-neural-network/cnn_1d_until_epoch_10.h5")
model